In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image_size = 28 # (width/length)
n_labels = 10 #  i.e. 0, 1, 2, 3, ..., 9 (10 digits)
image_pixels = image_size*image_size

data_path = "MNIST/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
                        delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
                       delimiter=",")

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data[:10]

In [ ]:
fac = 0.99 / 255
train_imgs = np.asfarray(train_data[:, 1:]) * fac + 0.01
test_imgs = np.asfarray(test_data[:, 1:]) * fac + 0.01

train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])

In [ ]:
train_labels[:]

### Converting the labels into one-hot representation

In [ ]:
digit = np.arange(10)

for label in range(10):
    one_hot = (digit==label).astype(int)
    print("label: ", label, " in one-hot representation: ", one_hot)

In [ ]:
digit = np.arange(n_labels)

# transform labels into one hot representation
train_labels_one_hot = (digit==train_labels).astype(float)
test_labels_one_hot = (digit==test_labels).astype(float)

# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

In [ ]:
train_labels_one_hot.shape

In [ ]:
train_labels_one_hot

In [ ]:
for i in range(15):
    img = train_imgs[i].reshape((28,28))
    plt.imshow(img, cmap="Greys")
    plt.show()

In [ ]:
import pickle

with open("pickled_mnist.pkl", "bw") as fh:
    data = (train_imgs, 
            test_imgs, 
            train_labels,
            test_labels)
    
    pickle.dump(data, fh)

In [ ]:
with open("pickled_mnist.pkl", "br") as fh:
    data = pickle.load(fh)

train_imgs = data[0]
test_imgs = data[1]
train_labels = data[2]
test_labels = data[3]

train_labels_one_hot = (digit==train_labels).astype(float)
test_labels_one_hot = (digit==test_labels).astype(float)

image_size = 28 # width and length
no_of_different_labels = 10 #  i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size

In [ ]:
train_imgs[0]

In [ ]:
from scipy.stats import truncnorm

def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

class MLP:
    
    def __init__(self, n_in, n_out, n_hidden, lr=0.1, bias=None):
        self.n_in = n_in # the number of input nodes 
        self.n_out = n_out # the number ot output nodes
        self.n_hidden = n_hidden # the number of hidden nodes
        self.lr = lr # the learning rate
        self.bias = bias
        
        self.weights()
        
    # a method to initialize the weight matrices (thetas)    
    def weights(self):
        
        bias_node = 1 if self.bias else 0 
        rad = 1 / np.sqrt(self.n_hidden + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        # the weights matrix connecting the input nodes with the hidden nodes
        self.w_in_hidden = X.rvs((self.n_hidden, self.n_in+bias_node))
        
        rad = 1 / np.sqrt(self.n_hidden + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        # the weights matrix connecting the hidden nodes with the output nodes
        self.w_hidden_out = X.rvs((self.n_out, self.n_hidden+bias_node))
        
        
    def _sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    # alternative activation function
    def ReLU(self,x):
        data = [max(0,value) for value in x]
        return np.array(data, dtype=np.float)

    # derivation of relu
    def ReLU_deriv(self,x):
        data = [1 if value>0 else 0 for value in x]
        return np.array(data, dtype=np.float)
    
    def train(self, in_vector, target):
        
        # make sure that the vectors have the right shape
        in_vector = np.array(in_vector)
        in_vector = in_vector.reshape(in_vector.size, 1)
        if self.bias:
            # adding bias node to the input_vector
            in_vector = np.concatenate( ([[self.bias]], in_vector) )
            
        target = np.array(target).reshape(target.size, 1)

        # the forward propagation
        in_hidden = self._sigmoid(self.w_in_hidden @ in_vector)
        #in_hidden = self.ReLU(self.w_in_hidden @ in_vector)
        # adding the bias node
        if self.bias:
            in_hidden = np.concatenate( ([[self.bias]], in_hidden) ) 
        output = self._sigmoid(self.w_hidden_out @ in_hidden)
        #output = self.ReLU(self.w_hidden_out @ in_hidden)
        
        # the error of the last layer
        out_error = target - output
        tmp = out_error * output * (1.0 - output)
        # tmp = out_error * self.ReLU_deriv(self.w_hidden_out @ in_hidden)
        self.w_hidden_out += self.lr  * (tmp @ in_hidden.T)

        # the error in the hidden layer:
        hidden_errors = self.w_hidden_out.T @ out_error # the backward propagation of the out error
        # update the weights:
        tmp = hidden_errors * in_hidden * (1.0 - in_hidden)
        # tmp = hidden_errors * self.ReLU_deriv(in_hidden)
        
        if self.bias:
            x = (tmp @in_vector.T)[1:,:]     # the first column cut off,
        else:
            x = tmp @ in_vector.T
            
        self.w_in_hidden += self.lr * x
        
        
    def run(self, in_vector):
        
        # make sure that input_vector is a column vector:
        in_vector = np.array(in_vector)
        in_vector = in_vector.reshape(in_vector.size, 1)
        # adding the bias node
        if self.bias:
            # adding bias node to the input_vector
            in_vector = np.concatenate( ([[self.bias]], in_vector) )
        
        # the forward propagation
        in_hidden = self._sigmoid(self.w_in_hidden @ in_vector)
        # in_hidden = self.ReLU(self.w_in_hidden @ in_vector)
        # adding the bias node
        if self.bias:
            in_hidden = np.concatenate( ([[self.bias]], in_hidden) ) 
        output = self._sigmoid(self.w_hidden_out @ in_hidden)
        # output = self.ReLU(self.w_hidden_out @ in_hidden)
        
        return output
    
    def evaluate(self, data, labels):
        # Counts how often the actual result corresponds to the target result.  
        # A result is considered to be correct, if the index of
        # the maximal value corresponds to the index with the "1"
        # in the one-hot representation:
        # res = [0.1, 0.132, 0.875]
        # labels[i] = [0, 0, 1]
        
        corrects, wrongs = 0, 0
        for i in range(len(data)):
            res = self.run(data[i])
            res_max = res.argmax()
            if res_max == labels[i].argmax():
                corrects += 1
            else:
                wrongs += 1
        return corrects, wrongs
    
    def confusion_matrix(self, data_array, labels):
        self.cm = np.zeros((10, 10), int)
        for i in range(len(data_array)):
            res = self.run(data_array[i])
            res_max = res.argmax()
            target = labels[i][0]
            self.cm[res_max, int(target)] += 1
        return self.cm 
    
    def precision(self, label):
        col = self.cm[:, label]
        return self.cm[label, label] / col.sum()
    
    def recall(self, label):
        row = self.cm[label, :]
        return self.cm[label, label] / row.sum()
    
    def accuracy(self):
        diagonal_sum = self.cm.trace()
        sum_of_all_elements = self.cm.sum()
        return diagonal_sum / sum_of_all_elements
    

In [ ]:
p=MLP(n_in=image_pixels, n_out=10, n_hidden=100, lr=0.1, bias=1)
    
for i in range(len(train_imgs)):
    p.train(train_imgs[i], train_labels_one_hot[i])

In [ ]:
res=p.run(test_imgs[4])
print(res)
print(np.max(res))
print(np.argmax(res))
print(test_labels_one_hot[4])
print(np.argmax(test_labels_one_hot[4]))

In [ ]:
for i in range(15):
    res = p.run(test_imgs[i])
    print(test_labels[i], np.argmax(res), np.max(res))

In [ ]:
cm = p.confusion_matrix(test_imgs, test_labels)
print(cm)

print("accuracy:", p.accuracy())

for i in range(10):
    print("digit: ", i, "precision: ", p.precision(i), "recall: ", p.recall(i))